Price and returns using Kalman Filter.

Generate prices based on some assumed price dynamics and then try uncover the patterns of speed. acceleration and variances.

$$
\Large
\begin{align}
P_{t+\delta} &= P_{t}  + \mu_t \delta + 0.5* a_t * \delta^2 + \epsilon_1\\
\mu_{t+\delta} &= \mu_t + a_t \delta + \epsilon_2\\
a_{t+\delta} &= a_t + \epsilon_3
\end{align}
$$

In [17]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

In [18]:
#! pip install filterpy

In [32]:
np.random.seed(144)

N = 200
a0 = 0.001
a_vals = [a0]
eps_3 = np.random.randn(N-1)/1000

a_vals = [a0]
for i in range(N-1):
    a_vals.append(a_vals[-1] +  eps_3[i])

a_vals = np.array(a_vals)
print(f"Length of avals = {len(a_vals)}")
a_vals[0:10]

Length of avals = 200


array([ 0.001     , -0.00029857, -0.00039111, -0.00032104,  0.00153402,
        0.00290428,  0.0027217 ,  0.00155167,  0.00257963,  0.00174516])

In [31]:
window_size = 
weights = np.ones(window_size) / window_size


AttributeError: module 'numpy' has no attribute 'rolling'

In [29]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter( 
                 x=np.array(range(N)),
                 y=a_vals,
                 name = "acceleration"))
fig.show()

##### DRIFT term

$$
\mu_{t+\delta} = \mu_t + a_t \delta + \epsilon_2
$$


In [23]:
eps_2 = np.random.randn(N)/10000

delta = 1/260

mu_0 = 0.10 * delta


mu_s = [mu_0]
for i in range(N):
    mu_s.append(mu_s[-1] + a_vals[i]*delta + eps_2[i])
    

In [6]:
fig = go.Figure()
fig.add_trace(go.Scatter( 
                 x=np.array(range(N)),
                 y=mu_s,
                 name = "mu values"))
fig.show()

#### prices as GBM (so that it remains non negative)

$$
\Large
\begin{align}
S_{t+\delta} &= S_t \cdot e^{( (\mu - 0.5\sigma^2))\delta + \sigma (B(t+\delta) - B(t))}\\
&=S_t \cdot e^{( (\mu - 0.5\sigma^2))\delta + \sigma z \sqrt \delta }
\end{align}
$$

above mu is yearly drift rate in theory, and $\mu \delta$ is daily drift or  $\mu_t$ by our definition. Also we keep $\sigma$ as annual volatility , lets say 20%

So price evolution becomes 
$$
\Large
\begin{align}
S_{t+\delta} &= S_t \cdot e^{( (\mu - 0.5\sigma^2))\delta + \sigma (B(t+\delta) - B(t))}\\
&=S_t \cdot e^{( (\mu - 0.5\sigma^2))\delta + \sigma z \sqrt \delta }\\
&=S_t \cdot e^{( (\mu \delta - 0.5 \sigma^2 \delta )) + \sigma z \sqrt \delta }\\
&=S_t \cdot e^{( (\mu_t - 0.5 \sigma^2 \delta )) + \sigma z \sqrt \delta }
\end{align}
$$


In [15]:
price_0 = 10.0
vol      = 0.10

zs = np.random.randn(N)

prices = [price_0]
for i in range(N):
    exp_term = np.exp(mu_s[i] - 0.5 *vol * vol *delta  + vol * np.sqrt(delta) *zs[i])
    prices.append(prices[-1] * exp_term)


In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter( 
                 x=np.array(range(N)),
                 y=prices,
                 name = "Prices"))
fig.show()

#### We have data now - we need to fit Kalman using our model


$$
\Large
\begin{align}
P_{t+\delta} &= P_{t}  + \mu_t \delta + 0.5* a_t * \delta^2 + \epsilon_1\\
\mu_{t+\delta} &= \mu_t + a_t \delta + \epsilon_2\\
a_{t+\delta} &= a_t + \epsilon_3
\end{align}
$$

We remodel this as change in prices $\Delta P_t = P_{t+\delta} -  P_{t}$

$$
\Large
\begin{align}
\Delta P_t &= \mu_t \delta + 0.5* a_t * \delta^2 + \epsilon_1\\
\mu_{t+\delta} &= \mu_t + a_t \delta + \epsilon_2\\
a_{t+\delta} &= a_t + \epsilon_3
\end{align}
$$





Writing as measurement and transition equations:

$$
\Large
\begin{align}
\Delta P_t &= \begin{bmatrix}
                \delta & 0.5\delta^2
              \end{bmatrix} \begin{bmatrix} \mu_t \\ a_t \end{bmatrix} + \epsilon_1\\
\begin{bmatrix} \mu_t \\ a_t \end{bmatrix} &=  \begin{bmatrix}
                                                    1 & \delta\\
                                                    0 & 1
                                                \end{bmatrix} \begin{bmatrix} \mu_{t-1} \\ a_{t-1} \end{bmatrix}   +  \epsilon  
\end{align}
$$

Assume We dont know $\mu_t, a_t$ and these are the hidden states we need to find out so that we can do predict and check. 

In [9]:
prices = np.array(prices)
delta_prices = (prices - np.roll(prices, 1))[1:]
#print(delta_prices[0:10])
#print(prices[0:10])

In [10]:
#1. start with theta0 as 
mu_guess = 0.0
acc_guess = 0.0

# How to estimate W, V ? using some MLE - 
W_t = np.diag([0.0001,0.0001])
V_t = vol * 1/delta
F_t = np.matrix([delta , 0.5 *delta *delta]).reshape(-1,2)

m0 = np.matrix([mu_guess,acc_guess]).reshape(2,-1)
m_prev = m0

#100 mean low confidence
C0 = np.matrix([[100, 0], [0, 100]])
C_prev = C0
G = np.matrix([[1.0, delta], [0, 1.0]])

posterior_states = []

for t in range(N-1):
    #2. One step forecast for the state
    #print(f"G: {G.shape} m_prev: {m_prev.shape}")
    a_t = G @ m_prev
    R_t = G @ C_prev @ G.T + W_t

    #3. One step forecast for the observation
    f_t = F_t@a_t
    Q_t = F_t @ R_t @ F_t.T + V_t

    #4. Compute Posterior
    y_t = delta_prices[t]
    #print(f"a_t: {a_t.shape} R_t: {R_t.shape} f_t:{f_t.shape} ")
    Q_t_inv = np.linalg.inv(Q_t)
    m_t = a_t + R_t @ (F_t.T) @ Q_t_inv * (y_t - f_t)
    C_t = R_t - R_t @ (F_t.T) @ Q_t_inv @ F_t @ R_t
    m_prev = m_t
    C_prev = C_t
    posterior_states.append(np.ravel(m_t))

In [11]:
posterior_mu = [m0[0,0]]
posterior_mu.extend([x[0] for x in posterior_states])
posterior_a = [m0[1,0]]
posterior_a.extend([x[1] for x in posterior_states])

In [12]:
len(posterior_mu)

200

In [13]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = np.array(range(N)), y = mu_s, name = "mu_s"))
fig.add_trace(go.Scatter(x = np.array(range(N)), y = posterior_mu, name = "posterior_mu_s"))
fig.show()

In [14]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = np.array(range(N)), y = a_vals, name = "a_vals"))
fig.add_trace(go.Scatter(x = np.array(range(N)), y = posterior_a, name = "posterior_a"))
fig.show()